# IoTアプリケーションの開発効率の向上を目的とした開発者によるコードの変更を部分適用する方式の提案と実装

このノートブックは、研究報告論文「IoTアプリケーションの開発効率の向上を目的とした開発者によるコードの変更を部分適用する方式の提案と実装」における提案方式の実装を、既存方式と比較検討した際のプロトコルを記録しておくためのものである。

計測方法について、北九州市立大学の山崎進氏による記事「[mix_tasks_upload_hotswap の Hexライブラリ版を試す](https://qiita.com/zacky1972/items/f0b47eded7c902008871)」がおおいに参考になりました。感謝いたします。

## 更新対象とするIoTアプリケーション

各方式の比較検討に用いるアプリケーションは、本リポジトリの[target_app](./target_app)ディレクトリに収められている。そのうちの、以下に抜粋したコードに含まれる`TargetApp.hello/0`メソッドが返す文字列を変更することで、コードの変更とする。

```
defmodule TargetApp do
  def hello do
    :world
  end
end
```

## 比較検討する方式

上記のIoTアプリケーションコードの変更をIoTデバイスに適用する方式として、以下の3つについて比較検討する。

1. ファームウェアイメージ全体の適用
2. ファームウェアイメージのパッチによる部分適用
3. アプリケーションコードの部分適用

それぞれの詳細については、研究報告論文を参照されたい。

## 比較検討の方法

各方式を比較するための指標として、開発者によるコードの変更をIoTデバイスに適用し、アプリケーションが通信可能となるまでの時間を用いる。

方式を実装したIoTデバイス開発プラットフォームとして、Nervesを用いる。筆者による提案方式3.の実装を加えると、上記の方式をすべて実装しているのがNervesだからである。ハードウェアとしては、Raspberry Piを用いる。

方式1.および2.については、スクリプト[measure_firmware_update.sh](./measure_firmware_update.sh)を用いて、以下の時間を計測する。

1. ファームウェアイメージの生成
2. ファームウェアイメージのデバイスへのアップロード
3. デバイスの再起動

方式3.については、提案方式を実装した`mix upload.hotswap`コマンドの実行時間を、`time(1)`によって計測する。

## 実験環境

### 開発環境

* MacBook Pro (13-inch, 2018, Four Thunderbolt 3 Ports)
* プロサッサ: 2.7 GHz クアッドコアIntel Core i7
* メモリ: 16 GB 2133 MHz LPDDR3

### IoTデバイス

* Raspberry Pi 3 Model B
* 開発環境との通信: 有線LAN

### Elixir/Nervesのバージョン

* Nerves: 1.7.1
* Nerves Bootstrap: 1.10.1
* Elixir: 1.11.2

このNotebookでは、Nervesの開発環境の整備については解説しない。「[ElixirでIoT#4.1：Nerves開発環境の準備（2020年11月版）](https://qiita.com/takasehideki/items/88dda57758051d45fcf9)」等の記事を参照されたい。

## 実験

### 1. ファームウェアイメージ全体の適用

スクリプト[measure_firmware_update.sh](./measure_firmware_update.sh)を用いて、以下に要する時間を計測する。

1. `mix firmware`
2. `mix upload`
3. Nervesが再起動して`ping(1)`が通るようになるまでの時間

3回計測を行い、それらの平均を用いる。

#### 1回目

In [4]:
!cd target_app && ../measure_firmware_update.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m21.927s
user	0m6.584s
sys	0m2.121s

Time to upload firmware

real	0m14.572s
user	0m5.182s
sys	0m1.261s

Time to reboot

time: 22.692 sec.



#### 2回目

In [1]:
!cd target_app && ../measure_firmware_update.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m20.573s
user	0m6.462s
sys	0m2.150s

Time to upload firmware

real	0m14.527s
user	0m5.599s
sys	0m1.337s

Time to reboot

time: 22.522 sec.



#### 3回目

In [5]:
!cd target_app && ../measure_firmware_update.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m21.081s
user	0m6.499s
sys	0m2.020s

Time to upload firmware

real	0m14.328s
user	0m5.159s
sys	0m1.214s

Time to reboot

time: 22.880 sec.



#### まとめ

In [11]:
print("1. ファームウェアイメージの生成: {:.2f}秒".format((21.927+20.573+21.081)/3))
print("2. ファームウェアイメージのデバイスへのアップロード: {:.2f}秒".format((14.572+14.527+14.328)/3))
print("3. デバイスの再起動: {:.2f}秒".format((22.692+22.522+22.880)/3))

1. ファームウェアイメージの生成: 21.19秒
2. ファームウェアイメージのデバイスへのアップロード: 14.48秒
3. デバイスの再起動: 22.70秒


### 2. ファームウェアイメージのパッチによる部分適用

スクリプト[measure_firmware_update.sh](./measure_firmware_update.sh)を用いて、以下に要する時間を計測する。

1. `mix firmware.patch`
2. `mix upload --firmware _build/rpi3_dev/nerves/images/patch.fw`
3. Nervesが再起動して`ping(1)`が通るようになるまでの時間

3回計測を行い、それらの平均を用いる。

#### 1回目

In [8]:
!cd target_app && ../measure_firmware_update.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m27.127s
user	0m6.697s
sys	0m2.227s

Time to upload firmware

real	0m17.873s
user	0m3.897s
sys	0m0.952s

Time to reboot

time: 22.534 sec.



#### 2回目

In [2]:
!cd target_app && ../measure_firmware_update.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m23.531s
user	0m6.589s
sys	0m2.185s

Time to upload firmware

real	0m18.240s
user	0m3.959s
sys	0m1.007s

Time to reboot

time: 22.743 sec.



#### 3回目

In [13]:
!cd target_app && ../measure_firmware_update.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m22.494s
user	0m6.610s
sys	0m2.180s

Time to upload firmware

real	0m17.922s
user	0m4.169s
sys	0m1.062s

Time to reboot

time: 22.638 sec.



#### まとめ

In [17]:
print("1. ファームウェアイメージの生成: {:.2f}秒".format((27.127+23.531+22.494)/3))
print("2. ファームウェアイメージのデバイスへのアップロード: {:.2f}秒".format((17.873+18.240+17.922)/3))
print("3. デバイスの再起動: {:.2f}秒".format((22.534+22.743+22.638)/3))

1. ファームウェアイメージの生成: 24.38秒
2. ファームウェアイメージのデバイスへのアップロード: 18.01秒
3. デバイスの再起動: 22.64秒


### 3. アプリケーションコードの部分適用

1. `mix upload.hotswap`

この方式ではNervesの再起動は必要ないため、適用の手順は以上のみである。上記のコマンドの実行に要する時間を、`time(1)`によって計測する。

3回計測を行い、それらの平均を用いる。

#### 1回目

In [11]:
!cd target_app && time mix upload.hotswap > /dev/null


real	0m3.414s
user	0m4.708s
sys	0m1.322s


#### 2回め

In [15]:
!cd target_app && time mix upload.hotswap > /dev/null


real	0m3.197s
user	0m4.668s
sys	0m1.256s


#### 3回目

In [16]:
!cd target_app && time mix upload.hotswap > /dev/null


real	0m2.915s
user	0m4.528s
sys	0m1.200s


#### まとめ

print("1. `mix upload.hotswap`: {:.2f}秒".format((3.414+3.197+2.915)/3))

#### 適用前後のデバイスとの通信を`ping(1)`で検証する

上記の適用を行う前後で、通信の遮断が発生するかどうかを、`ping(1)`コマンドにより検証する。

ターミナルを2つ開いておき、一方では以下のコマンドを用いて、`nerves.local`に対して0.1秒間隔で`ping(1)`コマンドを実行する（1）。

```
ping -i 0.1 nerves.local | while read pi; do echo "$(date '+[%Y/%m/%d %H:%M:%S]') $pi"; done
```

他方では、上述の`mix upload.hotswap`コマンドを`date(1)`コマンドを前後に挟むことで時間を表示して実行する（2）。

（1）の結果は以下の通りである。

```
$ ping -i 0.1 nerves.local | while read pi; do echo "$(date '+[%Y/%m/%d %H:%M:%S]') $pi"; done
[2021/01/15 16:54:24] PING nerves.local (172.31.68.221): 56 data bytes
[2021/01/15 16:54:24] 64 bytes from 172.31.68.221: icmp_seq=0 ttl=64 time=0.726 ms
[2021/01/15 16:54:24] 64 bytes from 172.31.68.221: icmp_seq=1 ttl=64 time=0.651 ms
[2021/01/15 16:54:24] 64 bytes from 172.31.68.221: icmp_seq=2 ttl=64 time=0.833 ms
[2021/01/15 16:54:24] 64 bytes from 172.31.68.221: icmp_seq=3 ttl=64 time=0.920 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=4 ttl=64 time=0.977 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=5 ttl=64 time=0.668 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=6 ttl=64 time=0.840 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=7 ttl=64 time=0.832 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=8 ttl=64 time=0.783 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=9 ttl=64 time=0.774 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=10 ttl=64 time=0.858 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=11 ttl=64 time=0.821 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=12 ttl=64 time=0.865 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=13 ttl=64 time=0.731 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=14 ttl=64 time=0.770 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=15 ttl=64 time=0.624 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=16 ttl=64 time=0.719 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=17 ttl=64 time=0.810 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=18 ttl=64 time=0.685 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=19 ttl=64 time=0.781 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=20 ttl=64 time=0.742 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=21 ttl=64 time=0.816 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=22 ttl=64 time=0.583 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=23 ttl=64 time=0.547 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=24 ttl=64 time=0.618 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=25 ttl=64 time=0.592 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=26 ttl=64 time=0.556 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=27 ttl=64 time=0.525 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=28 ttl=64 time=0.498 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=29 ttl=64 time=0.506 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=30 ttl=64 time=0.504 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=31 ttl=64 time=0.514 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=32 ttl=64 time=0.558 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=33 ttl=64 time=0.556 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=34 ttl=64 time=0.525 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=35 ttl=64 time=0.559 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=36 ttl=64 time=0.547 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=37 ttl=64 time=0.517 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=38 ttl=64 time=0.518 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=39 ttl=64 time=0.534 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=40 ttl=64 time=0.617 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=41 ttl=64 time=0.530 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=42 ttl=64 time=0.581 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=43 ttl=64 time=0.633 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=44 ttl=64 time=0.534 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=45 ttl=64 time=0.533 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=46 ttl=64 time=0.548 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=47 ttl=64 time=0.520 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=48 ttl=64 time=0.561 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=49 ttl=64 time=0.490 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=50 ttl=64 time=0.524 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=51 ttl=64 time=0.501 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=52 ttl=64 time=0.639 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=53 ttl=64 time=0.735 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=54 ttl=64 time=0.771 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=55 ttl=64 time=0.815 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=56 ttl=64 time=0.841 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=57 ttl=64 time=0.777 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=58 ttl=64 time=0.701 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=59 ttl=64 time=0.688 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=60 ttl=64 time=0.596 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=61 ttl=64 time=0.755 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=62 ttl=64 time=0.586 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=63 ttl=64 time=0.543 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=64 ttl=64 time=0.552 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=65 ttl=64 time=0.760 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=66 ttl=64 time=0.844 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=67 ttl=64 time=0.870 ms
^C
```

（2）の結果は以下の通りである。

```
$ date '+[%Y/%m/%d %H:%M:%S]'; mix upload.hotswap; date '+[%Y/%m/%d %H:%M:%S]'
[2021/01/15 16:54:26]
==> nerves
make[1]: Nothing to be done for `all'.
==> target_app

Nerves environment
  MIX_TARGET:   rpi3
  MIX_ENV:      dev

Compiling 2 files (.ex)
Generated target_app app
Successfully connected to hot_upload_test@nerves.local
Successfully deployed Elixir.TargetApp to hot_upload_test@nerves.local
Successfully deployed Elixir.TargetApp.Application to hot_upload_test@nerves.local
[2021/01/15 16:54:29]
```

よって、（1）の`ping(1)`の実行中に、（2）のコマンド実行による通信の切断は見られなかった。

### 各方式の計測結果のまとめ

|                         | firmware build | firmware upload | reboot | code update | sum   |
|-------------------------|----------------|-----------------|--------|-------------|-------|
| firmware update (full)  |          21.19 |           14.48 |  22.70 |           - | 58.37 |
| firmware update (patch) |          24.38 |           18.01 |  22.64 |           - | 65.04 |
| proposed method         |              - |               - |      - |        3.18 |  3.18 |